In [1]:
import numpy as np
import random_variate
import scipy

### Comparing Bernoulli RVs

#### Performance

In [19]:
RV_Bern = random_variate.Bernoulli(0.2)
scipy_bern = scipy.stats.bernoulli

In [20]:
%%timeit
scipy_bern_rvs = scipy_bern.rvs(0.2, size=1)

11.7 µs ± 83.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [21]:
%%timeit
RV_Bern.generate_random()

55.1 ns ± 0.184 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Interestingly, our implementation is actually faster for generating one variable. However, we see that scipy is faster for generating multiple variables.

In [22]:
%%timeit
scipy_bern_rvs = scipy_bern.rvs(0.2, size=10000)

108 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
%%timeit
l = [RV_Bern.generate_random() for _ in range(10000)]

666 µs ± 4.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


#### Goodness of Fit


To test the Bernoulli distribution we can use the Binomial test available from SciPy

In [74]:
results = []
for _ in range(500):
    l = np.array([RV_Bern.generate_random() for _ in range(10000)])
    Y = sum(l)
    pval = scipy.stats.binomtest(Y, 10000, 0.2).pvalue
    results.append(pval)

In [79]:
len([i for i in results if i < 0.05]) 

21

We see that in all but 21 of our runs, we accept the null hypothesis that the distributions are the same, at a 95% confidence interval. Intuitively, we expect that we reject the null hypothesis about 5% of the time, as seen here.

### Comparing Geometric RVs

#### Performance

In [11]:
rng = np.random.default_rng(12345)
Geom = random_variate.Geometric(0.3)

In [12]:
%%timeit
rng.geometric(p=0.3, size = 1)

385 ns ± 0.91 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%%timeit
Geom.generate_random()

1.21 µs ± 1.91 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Unsurprisingly, numpy is faster than our generator, but the difference is within an order of magnitude.

In [14]:
%%timeit
rng.geometric(p=0.3, size = 10000)

79.8 µs ± 34.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
%%timeit
[Geom.generate_random() for _ in range (10000)]

12.2 ms ± 63.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Again, we see more than an order of magnitude difference in performance once the populations generated become large.

#### Goodness of Fit

Now, let's use the Kolomogorov-Smirnov test, implemented via the scipy library, to test whether our implementation is a good fit for the distribution generated by numpy.

We will generate 10,000 geometric RVs using numpy and our library, and perform a K-S test on the two generated sets of RVs. We will repeat this 500 times.

In [16]:
results = []
for i in range(500):
    numpy_geoms = rng.geometric(p=0.3, size = 10000)
    RV_geoms = [Geom.generate_random() for _ in range (10000)]
    result = scipy.stats.kstest(numpy_geoms, RV_geoms)
    results.append(result)

In [17]:
print(len([r for r in results if r.pvalue >.05])) # check if we reject the null hypothesis in any runs
print(np.mean([r.pvalue for r in results])) # check the mean p-value

493
0.7736723811960008


We see that in most of our runs (493) we fail to reject the null hypothesis that the distributions are the same. However, in a few of the runs we do reject the null hypothesis, likely reflecting random variation among our generated population (by numpy) and our own populations.

### Comparing Exponential RVs

In [18]:
Exp = random_variate.Exponential(1)


In [19]:
%%timeit
rng.exponential(scale=1, size=1)

393 ns ± 0.683 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [20]:
%%timeit
Exp.generate_random()

423 ns ± 3.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


With one random variate, both methods are close

In [21]:
%%timeit
rng.exponential(scale=1, size=10000)

48 µs ± 128 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
%%timeit
[Exp.generate_random() for _ in range(10000)]

4.29 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As with the other native random variate methods, we see greater than an order of magnitude difference (here, about two orders of magnitude) in speed. We can see that the Numpy method scales more efficiently than our method, which scales linearly.

#### Goodness of Fit

Now, let's use the Kolomogorov-Smirnov test, implemented via the scipy library, to test whether our implementation is a good fit for the distribution generated by scipy.

We will generate 10,000 Exponential RVs using scipy and our library, and perform a K-S test on the two generated sets of RVs. We will repeat this 500 times.

In [23]:
results = []
for i in range(500):
    numpy_exp = rng.exponential(scale=1, size = 10000)
    RV_exp = [Exp.generate_random() for _ in range (10000)]
    result = scipy.stats.kstest(numpy_exp, RV_exp)
    results.append(result)

In [24]:
print(len([r for r in results if r.pvalue >.05])) # check if we reject the null hypothesis in any runs
print(np.mean([r.pvalue for r in results])) # check the mean p-value

475
0.5138691559785878


We see that in most of our runs (475, 95%) we fail to reject the null hypothesis that the distributions are the same. However, in 25 of the runs we do reject the null hypothesis, likely reflecting random variation among our generated population (by numpy) and our own populations.

### Uniform Random Variates

#### Performance

In [25]:
Unif = random_variate.Uniform(0,1)

In [27]:
%%timeit
Unif.generate_random()

128 ns ± 0.724 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [28]:
%%timeit
rng.uniform(low=0, high=1)

913 ns ± 2.87 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [30]:
%%timeit
[Unif.generate_random() for _ in range(10000)]

1.44 ms ± 6.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [31]:
%%timeit
rng.uniform(low=0, high=1, size=10000)

41.1 µs ± 52.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Again, we see the performance difference, well more than order of magnitude, between our generator and the NumPy generator, which does not scale linearly.

In [32]:
results = []
for i in range(500):
    numpy_unif = rng.uniform(low=0, high=1, size=10000)
    RV_unif = [Unif.generate_random() for _ in range(10000)]
    result = scipy.stats.kstest(numpy_unif, RV_unif)
    results.append(result)

In [33]:
print(len([r for r in results if r.pvalue >.05])) # check if we reject the null hypothesis in any runs
print(np.mean([r.pvalue for r in results])) # check the mean p-value

480
0.5102099156888605


Again, we see that in over 95% of runs (here, 96%) we fail to reject the null hypothesis.

### Weibull

In [34]:
Weib = random_variate.Weibull(lam=1, k=2)

In [35]:
%%timeit
Weib.generate_random()

537 ns ± 8.06 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [36]:
%%timeit
rng.weibull(2,1)

400 ns ± 1.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [37]:
%%timeit
[Weib.generate_random() for _ in range(10000)]

5.45 ms ± 21 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [38]:
%%timeit
rng.weibull(2,10000)

124 µs ± 2.67 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


As with the other generators, while our generator is close in performance to NumPy for generating a single RV, it is not competitive for generating large quantities of RVs.

In [61]:
results = []
for i in range(500):
    numpy_weib = rng.weibull(2,10000)
    RV_weib = [Weib.generate_random() for _ in range(10000)]
    result = scipy.stats.kstest(numpy_weib, RV_weib)
    results.append(result)

In [62]:
print(len([r for r in results if r.pvalue >.05])) # check if we reject the null hypothesis in any runs
print(np.mean([r.pvalue for r in results])) # check the mean p-value

487
0.5123926049162391


Again, we see that in well over 90% of runs, we fail to reject the null hypothesis that our generator is different from the NumPy generator.

## Normal

In [2]:
Norm = random_variate.Normal(1,1)

In [3]:
rng = np.random.default_rng(12345)

In [4]:
%%timeit
Norm.generate_random()

1.18 µs ± 2.86 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [5]:
%%timeit
rng.normal(1,1)

364 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
%%timeit
[Norm.generate_random() for _ in range(10000)]

12.2 ms ± 61 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit
rng.normal(1,1,10000)

47.3 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [8]:
results = []
for i in range(500):
    numpy_norm = rng.normal(1,1,10000)
    RV_norm = [Norm.generate_random() for _ in range(10000)]
    result = scipy.stats.kstest(numpy_norm, RV_norm)
    results.append(result)

In [9]:
print(len([r for r in results if r.pvalue >.05])) # check if we reject the null hypothesis in any runs
print(np.mean([r.pvalue for r in results])) # check the mean p-value

486
0.5155199987303636


## Erlang

In [10]:
Erlang = random_variate.Erlang(1,1)

In [11]:
%%timeit
Erlang.generate_random()

1.89 µs ± 8.29 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [12]:
%%timeit
rng.gamma(1,1)

368 ns ± 0.74 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [13]:
%%timeit
[Erlang.generate_random() for _ in range (10000)]

19.2 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
rng.gamma(1,1,10000)

48.7 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
results = []
for i in range(500):
    numpy_erl = rng.gamma(1,1,10000)
    RV_erl = [Erlang.generate_random() for _ in range(10000)]
    result = scipy.stats.kstest(numpy_erl, RV_erl)
    results.append(result)

In [18]:
print(len([r for r in results if r.pvalue >.05])) # check if we reject the null hypothesis in any runs
print(np.mean([r.pvalue for r in results])) # check the mean p-value

478
0.5041129476208269


### Triangular

In [81]:
Tria = random_variate.Triangular(1,2,3)

In [82]:
%%timeit
[Tria.generate_random() for i in range(10_000)]

5.26 ms ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
%%timeit
rng.triangular(1,2,3, 10000)

41.2 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [86]:
results = []
for i in range(500):
    numpy_tria = rng.triangular(1,2,3, 10000)
    RV_tria = [Tria.generate_random() for _ in range(10000)]
    result = scipy.stats.kstest(numpy_tria, RV_tria)
    results.append(result)

In [87]:
print(len([r for r in results if r.pvalue >.05])) 

481
